# Semantic Kernel dengan Integrasi Server OpenBnB MCP

Notebook ini menunjukkan cara menggunakan Semantic Kernel dengan server OpenBnB MCP yang sebenarnya untuk mencari akomodasi Airbnb nyata menggunakan MCPStdioPlugin. Untuk Akses LLM, ini menggunakan Microsoft Foundry. Untuk mengatur variabel lingkungan Anda, Anda dapat mengikuti [Pelajaran Setup](/00-course-setup/README.md)


## Impor Paket yang Dibutuhkan


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Membuat Koneksi Plugin MCP

Kita akan terhubung ke [OpenBnB MCP server](https://github.com/openbnb-org/mcp-server-airbnb) menggunakan MCPStdioPlugin. Server ini menyediakan fungsi pencarian Airbnb melalui paket @openbnb/mcp-server-airbnb.


## Membuat Klien

Dalam contoh ini, kami akan menggunakan Microsoft Foundry untuk akses LLM kami. Pastikan variabel lingkungan Anda sudah diatur dengan benar.


## Konfigurasi Lingkungan

Konfigurasikan pengaturan Azure OpenAI. Pastikan Anda telah mengatur variabel lingkungan berikut:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Memahami Integrasi OpenBnB MCP

Notebook ini terhubung ke **server OpenBnB MCP asli** yang menyediakan fungsi pencarian Airbnb yang sebenarnya.

### Cara kerjanya:

1. **MCPStdioPlugin**: Menggunakan komunikasi input/output standar dengan server MCP  
2. **Paket NPM Asli**: Mengunduh dan menjalankan `@openbnb/mcp-server-airbnb` melalui npx  
3. **Data Langsung**: Mengembalikan data properti Airbnb yang sebenarnya dari API mereka  
4. **Penemuan Fungsi**: Agen secara otomatis menemukan fungsi yang tersedia dari server MCP  

### Fungsi yang Tersedia:

Server OpenBnB MCP biasanya menyediakan:  
- **search_listings** - Mencari properti Airbnb berdasarkan lokasi dan kriteria  
- **get_listing_details** - Mendapatkan informasi detail tentang properti tertentu  
- **check_availability** - Memeriksa ketersediaan untuk tanggal tertentu  
- **get_reviews** - Mengambil ulasan untuk properti  
- **get_host_info** - Mendapatkan informasi tentang tuan rumah properti  

### Prasyarat:

- **Node.js** terpasang di sistem Anda  
- **Koneksi internet** untuk mengunduh paket server MCP  
- **NPX** tersedia (disertakan dengan Node.js)  

### Menguji Koneksi:

Anda dapat menguji server MCP secara manual dengan menjalankan:  
```bash
npx -y @openbnb/mcp-server-airbnb
```
  
Ini akan mengunduh dan memulai server OpenBnB MCP, yang kemudian dihubungkan oleh Semantic Kernel untuk data Airbnb yang sebenarnya.


## Menjalankan Agen dengan Server OpenBnB MCP

Sekarang kita akan menjalankan Agen AI yang terhubung ke server OpenBnB MCP untuk mencari akomodasi Airbnb nyata di Stockholm untuk 2 dewasa dan 1 anak. Silakan ubah daftar `user_inputs` untuk memodifikasi kriteria pencarian.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# Ringkasan
Selamat! Anda telah berhasil membangun agen AI yang terintegrasi dengan pencarian akomodasi dunia nyata menggunakan Model Context Protocol (MCP):

## Teknologi yang Digunakan:
- Semantic Kernel - Untuk membangun agen cerdas dengan Azure OpenAI
- Microsoft Foundry - Untuk kemampuan LLM dan penyelesaian obrolan
- MCP (Model Context Protocol) - Untuk integrasi alat yang terstandarisasi
- OpenBnB MCP Server - Untuk fungsionalitas pencarian Airbnb nyata
- Node.js/NPX - Untuk menjalankan server MCP eksternal

## Apa yang Telah Anda Pelajari:
- Integrasi MCP: Menghubungkan agen Semantic Kernel ke server MCP eksternal
- Akses Data Real-time: Mencari properti Airbnb nyata melalui API langsung
- Komunikasi Protokol: Menggunakan komunikasi stdio antara agen dan server MCP
- Penemuan Fungsi: Secara otomatis menemukan fungsi yang tersedia dari server MCP
- Streaming Respons: Menangkap dan mencatat panggilan fungsi secara real-time
- Rendering HTML: Memformat respons agen dengan tabel bergaya dan tampilan interaktif

## Langkah Selanjutnya:
- Mengintegrasikan server MCP tambahan (cuaca, penerbangan, restoran)
- Membangun sistem multi-agen yang menggabungkan protokol MCP dan A2A
- Membuat server MCP kustom untuk sumber data Anda sendiri
- Menerapkan memori percakapan persisten lintas sesi
- Mendeploy agen ke Azure Functions dengan orkestrasi server MCP
- Menambahkan autentikasi pengguna dan kemampuan pemesanan


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan layanan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Meskipun kami berupaya untuk mencapai ketepatan, harap diketahui bahwa terjemahan otomatis mungkin mengandung kesalahan atau ketidakakuratan. Dokumen asli dalam bahasa aslinya harus dianggap sebagai sumber yang sahih. Untuk informasi penting, disarankan menggunakan terjemahan profesional oleh manusia. Kami tidak bertanggung jawab atas kesalahpahaman atau salah tafsir yang timbul dari penggunaan terjemahan ini.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
